# Make predictions using the Rephetio algorithm
* This notebook follows the approach of the [MechRepoNet paper](https://github.com/SuLab/MechRepoNet/tree/d219c7e2540513c2338d6623348697142a08f73a)
* Please generate the `nodes_biolink.csv` and `edges_biolink.csv` from the original code

* This notebook executes the following steps:
    * Generate a holdout set from the 'test' and 'valid' splits
    * do some preprocessing of the data
    * Find best hyperparameters
    * Make predictions
    
* running the entire notebook can take up to a day

In [1]:
# dataset generation
import pandas as pd
import os

# prerequisites
from tqdm import tqdm
from copy import deepcopy

# hpo
from pathlib import Path
import re as regex
import pickle
import numpy as np
from time import time

import re as regex
from itertools import chain
import data_tools.graphs as gt
from hetnet_ml.extractor import MatrixFormattedGraph, piecewise_extraction

%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
from data_tools.plotting import count_plot_h

# test

/home/rogertu/miniforge3/envs/kge/lib/python3.11/site-packages/data_tools/df_processing.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
# from the relative directory get to the root
os.chdir("./Consilience-Drug-Repurposing")

## Generate a holdout set for MIND from original MRN dataset
* MIND is in a triple format
* MRN is in a Neo4j format (property tables)
* In this section, we shoe horn MIND into the rephetio algorithm by mapping MIND edges to MRN.

### import in files
* nodes/edges
* MIND

In [3]:
# import MIND
mind_edges = pd.concat(
    [
        pd.read_csv("./data/MIND/train.txt", sep="\t", names=["h", "r", "t"]).assign(
            label="train"
        ),
        pd.read_csv("./data/MIND/valid.txt", sep="\t", names=["h", "r", "t"]).assign(
            label="valid"
        ),
        pd.read_csv("./data/MIND/test.txt", sep="\t", names=["h", "r", "t"]).assign(
            label="test"
        ),
    ]
)

# import biolink data from the data folder. This is from MechRepoNet
nodes = pd.read_csv("./data/nodes_biolink.csv", dtype=str)
edges = pd.read_csv("./data/edges_biolink.csv", dtype=str)

### Generate nodes and edges file
* filter imported nodes file from the concatenation of the head and tail of the MRN edges 
* get appropriate edge types for every edge in MRN

#### Generate the nodes file

In [5]:
mind_nodes = list(set(mind_edges["h"]).union(set(mind_edges["t"])))

# all nodes in mind are nested in the nodes file.
mind_nodes = nodes.query("id in @mind_nodes")[["id", "name", "label"]]

print(f"Number of nodes: {len(mind_nodes):,}")
print(f"All nodes in MIND: {mind_nodes.shape[0]==249605}")

Number of nodes: 249,605


### Generate the edges file
* export both nodes and edges

In [7]:
mind_edges.head()

,h,r,t,label
0,UNII:BTY153760O,inhibits_CinG,NCBIGene:3605,train
1,CHEBI:10056,activates_CaG,NCBIGene:1129,train
2,CHEBI:10056,activates_CaG,NCBIGene:1131,train
3,CHEBI:10056,activates_CaG,NCBIGene:1133,train
4,CHEBI:10056,activates_CaG,NCBIGene:3350,train


### add metapath info to indication edge

In [22]:
abbrev = {
    "AnatomicalEntity": "A",
    "BiologicalProcessOrActivity": "BP",
    "MacromolecularMachine": "G",
    "ChemicalSubstance": "C",
    "Disease": "D",
    "PhenotypicFeature": "P",
    "Pathway": "PW",
    "GeneFamily": "F",
    "OrganismTaxon": "T",
}

In [9]:
# split by indication and non-indication
mind_edges_not_ind = mind_edges[~mind_edges["r"].str.contains("indication")]
mind_edges_ind = mind_edges[mind_edges["r"].str.contains("indication")]

In [30]:
# double merge your nodes to get the labels
mind_edges_ind = (
    # get h_label
    mind_edges_ind.merge(mind_nodes, left_on="h", right_on="id", how="left")[
        ["h", "r", "t", "label_x", "label_y"]
    ]
    # get t_label
    .merge(mind_nodes, left_on="t", right_on="id", how="left").rename(
        columns={
            "label": "t_lab",
            "label_x": "label",
            "label_y": "h_lab",
        }
    )[["h", "r", "t", "label", "h_lab", "t_lab"]]
)
mind_edges_ind.head()

,h,r,t,label,h_lab,t_lab
0,CHEBI:32120,indication,REACT:R-HSA-2160456,train,ChemicalSubstance,Pathway
1,UBERON:0001132,indication,DOID:11199,train,AnatomicalEntity,Disease
2,CHEBI:10023,indication,DOID:0050289,train,ChemicalSubstance,Disease
3,CHEBI:100241,indication,DOID:12385,train,ChemicalSubstance,Disease
4,CHEBI:100241,indication,DOID:13258,train,ChemicalSubstance,Disease


In [33]:
# translate using the abbrevation dictionary
mind_edges_ind["h_lab"] = mind_edges_ind["h_lab"].map(lambda x: abbrev[x])
mind_edges_ind["t_lab"] = mind_edges_ind["t_lab"].map(lambda x: abbrev[x])

# remake r
mind_edges_ind["r"] = (
    "indication" + "_" + mind_edges_ind["h_lab"] + "i" + mind_edges_ind["t_lab"]
)

mind_edges_ind.head()

In [38]:
# restack indications and non-indications
mind_edges = pd.concat([mind_edges_not_ind, mind_edges_ind[["h", "r", "t", "label"]]])
mind_edges.tail()

,h,r,t,label
5365,CHEBI:9667,indication_CiG,NCBIGene:367,test
5366,CHEBI:41423,indication_CiG,NCBIGene:367,test
5367,CHEBI:9168,indication_CiG,NCBIGene:7490,test
5368,CHEBI:41879,indication_CiG,NCBIGene:367,test
5369,CHEBI:8378,indication_CiG,NCBIGene:367,test


In [39]:
mind_nodes.to_csv("./data/nodes_rep.csv", index=False)
mind_edges.rename(columns={"h": "start_id", "r": "type", "t": "end_id"}, inplace=True)
mind_edges[["start_id", "type", "end_id"]].to_csv("./data/edges_rep.csv", index=False)

### Prepare holdout set

In [40]:
# holdout is test
remain_edges_no_test = mind_edges.query('label!="test"')[["start_id", "type", "end_id"]]
remain_edges_no_test.to_csv("./data/remain_edges_no-test.csv", index=False)

# export test holdout
holdout_edges_test = mind_edges.query('label=="test"')[["start_id", "type", "end_id"]]
holdout_edges_test.to_csv("./data/holdout_edges_test.csv", index=False)

# holdout is valid
remain_edges_no_valid = mind_edges.query('label!="valid"')[
    ["start_id", "type", "end_id"]
]
remain_edges_no_valid.to_csv("./data/remain_edges_no-valid.csv", index=False)

# export valid holdout
holdout_edges_valid = mind_edges.query('label=="valid"')[["start_id", "type", "end_id"]]
holdout_edges_valid.to_csv("./data/holdout_edges_valid.csv", index=False)

## Generate metapath files (prerequisites for HPO and testing)
* To run rephetio, metapath counts must first be calculated
* We calculate the metapath counts for:
    * all metapaths
    * all positive metapaths (known trues)
    * all idealized training metapaths (metapaths from DrugMechDB v1)
    * all negative metapaths (non-trues)

#### get edge info and add indications to it

In [3]:
# get edge information
e_info = pd.read_csv(
    "/home/rogertu/projects/Consilience-Drug-Repurposing/path-based/rephetio/0_data/manual/edge_semtypes.csv"
)

# add indication edge information
add_indication = pd.DataFrame(
    {
        "fwd_edge": ["indication"],
        "abbrev": ["i"],
        "rev_edge": ["indication_of"],
        "rel_dir": [-1],
        "directed": [True],
        "parent_rel": [np.nan],
    }
)

# create new edge information
e_info = pd.concat([e_info, add_indication])
directed_map = e_info.set_index("fwd_edge")["directed"]

#### Get metapaths

In [4]:
mind_nodes = pd.read_csv("./data/nodes_rep.csv")
remain_edges_no_test = pd.read_csv("./data/remain_edges_no-test.csv")
remain_edges_no_valid = pd.read_csv("./data/remain_edges_no-valid.csv")

In [5]:
# generate the metapaths

# edges are the remaining edges, no test or valid
nodes_ls, edges_ls, mps_ls = list(), list(), list()

# add nodes and edges
nodes_ls.append(mind_nodes)
nodes_ls.append(mind_nodes)
edges_ls.append(remain_edges_no_test)
edges_ls.append(remain_edges_no_valid)

# create the metapath files
for i, v in enumerate(edges_ls):
    mps = gt.dataframes_to_metagraph(nodes_ls[i], v).extract_metapaths(
        "ChemicalSubstance", "Disease", 4
    )
    mps = [mp for mp in mps if len(mp) > 1]
    mps_ls.append(mps)

#### Look at similarity and non-similarity based paths

In [6]:
# look for similarity based paths
similarity_paths_check_ls = list()
for mps in mps_ls:
    similarity_paths_check = []

    for mp in tqdm(mps):
        if len(mp) < 2:
            similarity_paths_check.append(True)
            continue
        this_path = []
        for label in ["ChemicalSubstance"]:
            this_path.append(gt.is_similarity(mp, ["ChemicalSubstance"], directed_map))
        this_path.append(gt.is_similarity(mp, "Disease", directed_map, max_repeats=1))

        # Don't want CtXaYzD edges.. Ct (and Cm) edges should be banned...
        # Though not strictly similarity edges, they have heavy implications for Disease Phenotype similarity
        bl_edge = ["Ct", "Cm", "Cdg", "Cpl", "Cpv", "Ci"]
        for bl in bl_edge:
            if mp.abbrev.startswith(bl) or mp.abbrev.endswith("aw" + bl + "D"):
                this_path.append(True)

        similarity_paths_check.append(sum(this_path) > 0)

    similarity_paths_check_ls.append(similarity_paths_check)

  0%|                                                                                                     | 0/30567 [00:00<?, ?it/s]

100%|██████████████████████████████████████████████████████████████████████████████████████| 26484/26484 [00:01<00:00, 22939.31it/s]


In [7]:
# get non-similarity paths
non_sim_mps_ls = [
    [mp for mp, sim in zip(mps, similarity_paths_check) if not sim] for mps in mps_ls
]
[
    print(f"Unique metapaths{i}: {len(non_sim_mps):,}")
    for i, non_sim_mps in enumerate(non_sim_mps_ls)
]

Unique metapaths0: 8,198
Unique metapaths1: 8,198


[None, None]

In [8]:
for i, non_sim_mps in enumerate(non_sim_mps_ls):
    print(
        f"{len(non_sim_mps)/len(mps_ls[i]):.2%} of all metapaths are not similarity based"
    )

26.82% of all metapaths are not similarity based
30.95% of all metapaths are not similarity based


#### Generate metagraph

In [9]:
print("Generating graph ...")
mg_ls = [
    MatrixFormattedGraph(
        nodes_ls[i],
        edges_ls[i],
        "ChemicalSubstance",
        "Disease",
        max_length=4,
        w=0.4,
        n_jobs=30,
    )
    for i, nodes in enumerate(nodes_ls)
]

print("Done.")

Generating graph ...
Processing node and edge data...
Initializing metagraph...
Generating adjacency matrices...


100%|███████████████████████████████████████████████████████████████████████████████████████████████| 75/75 [00:52<00:00,  1.42it/s]



Determining degrees for each node and metaedge


100%|███████████████████████████████████████████████████████████████████████████████████████████████| 75/75 [00:25<00:00,  2.92it/s]



Weighting matrices by degree with dampening factor 0.4...


...


Determining degrees for each node and metaedge


100%|███████████████████████████████████████████████████████████████████████████████████████████████| 74/74 [00:26<00:00,  2.84it/s]



Weighting matrices by degree with dampening factor 0.4...


100%|███████████████████████████████████████████████████████████████████████████████████████████████| 74/74 [00:00<00:00, 84.66it/s]


Done.


In [10]:
# estimate the memory required to extract all compound-disease pairs
float_size = 32  # bits
bits_per_gb = 8589934592


def print_mem_info(n_comp, n_dis, n_mps):
    print(f"{n_comp:,} Compounds * {n_dis:,} Diseases = {n_comp * n_dis:,} C-D Pairs")
    print(
        f"{n_comp * n_dis:,} C-D Pairs * {n_mps:,} Metapaths = {n_comp * n_dis * n_mps:,} Matrix Values"
    )
    print(
        f"{n_comp * n_dis * n_mps * float_size / (bits_per_gb):1,.1f} GB of matrix values"
    )
    print(f"{n_comp * n_dis * float_size / (bits_per_gb):1,.3f} GB per metapath")

In [11]:
# look at total compounds and diseases and estimate metapath memory usage
total_comps_ls, total_dis_ls = list(), list()
for i, nodes in enumerate(nodes_ls):
    total_comps = nodes["label"].value_counts()["ChemicalSubstance"]
    total_dis = nodes["label"].value_counts()["Disease"]

    total_comps_ls.append(total_comps)
    total_dis_ls.append(total_dis)
    print_mem_info(total_comps, total_dis, len(non_sim_mps_ls[i]))
    print("")

40,455 Compounds * 13,942 Diseases = 564,023,610 C-D Pairs
564,023,610 C-D Pairs * 8,198 Metapaths = 4,623,865,554,780 Matrix Values
17,225.2 GB of matrix values
2.101 GB per metapath

40,455 Compounds * 13,942 Diseases = 564,023,610 C-D Pairs
564,023,610 C-D Pairs * 8,198 Metapaths = 4,623,865,554,780 Matrix Values
17,225.2 GB of matrix values
2.101 GB per metapath



In [12]:
def piecewise_extraction(function, to_split, block_size=1000, **params):

    assert type(to_split) == str and to_split in params

    # Won't want progress bars for each subsetx
    params["verbose"] = False

    # Retain a copy of the original parameters
    full_params = deepcopy(params)
    total = len(params[to_split])

    # Determine the number of iterations needed
    num_iter = total // block_size
    if total % block_size != 0:
        num_iter += 1

    all_results = []
    for i in tqdm(range(num_iter)):

        # Get the start and end indicies
        start = i * block_size
        end = (i + 1) * block_size

        # End can't be larger than the total number items
        if end > total:
            end = total

        # Subset the paramter of interest
        params[to_split] = full_params[to_split][start:end]

        # Get the funciton results
        all_results.append(function(**params))

    return pd.concat(all_results, sort=False, ignore_index=True)

### Get metapath pair counts for each metapath in both test and valid

In [13]:
# This can take like 4 hours
mp_counts_ls = list()

print(f"Extracting metapath pair counts ...")

for i, mps in enumerate(mps_ls):
    file_loc = os.path.join("./path-based/rephetio/0_data", f"mp_counts0_{i}.pkl")

    if os.path.exists(file_loc):
        with open(file_loc, "rb") as f:
            mp_counts = pickle.load(f)

        mp_counts_ls.append(mp_counts)

    else:
        mp_counts = piecewise_extraction(
            function=mg_ls[i].extract_metapath_pair_counts,
            to_split="metapaths",
            block_size=200,
            metapaths=[mp.abbrev for mp in mps],
            start_nodes="ChemicalSubstance",
            end_nodes="Disease",
            n_jobs=30,
        )
        mp_counts["subset"] = "all_pairs"
        mp_counts["frac"] = (
            mp_counts["pair_count"] / total_comps_ls[i] * total_dis_ls[i]
        )

        mp_counts_ls.append(mp_counts)
        with open(file_loc, "wb") as f:
            pickle.dump(mp_counts, f)

print("Done.")

Extracting metapath pair counts ...
Done.


In [14]:
# get paths populated for each metapath object
for i, mp_counts in enumerate(mp_counts_ls):
    print(
        f'Percent of paths populated for {i} object: {(mp_counts["pair_count"] != 0).sum() / len(mp_counts):1.2%}'
    )

Percent of paths populated for 0 object: 70.18%
Percent of paths populated for 1 object: 75.45%


### Get positive metapaths and training positives

In [15]:
# All potential training Positives
keep_comps_ls = [
    set(edges.query('type == "treats_CtD" | type == "indication_CiD"')["start_id"])
    for edges in edges_ls
]

keep_dis_ls = [
    set(edges.query('type == "treats_CtD"| type=="indication_CiD"')["end_id"])
    for edges in edges_ls
]

print("All potential training positives")
for i, keep_comps in enumerate(keep_comps_ls):
    print_mem_info(len(keep_comps), len(keep_dis_ls[i]), len(non_sim_mps_ls[i]))

All potential training positives
11,352 Compounds * 5,631 Diseases = 63,923,112 C-D Pairs
63,923,112 C-D Pairs * 8,198 Metapaths = 524,041,672,176 Matrix Values
1,952.2 GB of matrix values
0.238 GB per metapath
11,350 Compounds * 5,627 Diseases = 63,866,450 C-D Pairs
63,866,450 C-D Pairs * 8,198 Metapaths = 523,577,157,100 Matrix Values
1,950.5 GB of matrix values
0.238 GB per metapath


In [16]:
# This can take about 5 hours
print(f"Extracting positive metapath pair counts ...")

mp_counts_ls2 = list()
for i, mg in enumerate(mg_ls):
    file_loc = os.path.join("./path-based/rephetio/0_data", f"mp_counts1_{i}.pkl")

    if os.path.exists(file_loc):
        with open(file_loc, "rb") as f:
            mp_counts = pickle.load(f)

        mp_counts_ls.append(mp_counts)
    else:

        mp_counts = piecewise_extraction(
            function=mg.extract_metapath_pair_counts,
            to_split="metapaths",
            block_size=500,
            metapaths=[mp.abbrev for mp in mps_ls[i]],
            start_nodes=list(keep_comps_ls[i]),
            end_nodes=list(keep_dis_ls[i]),
            n_jobs=30,
        )
        mp_counts["subset"] = "all_pos"
        mp_counts["frac"] = mp_counts["pair_count"] / (
            len(keep_comps_ls[i]) * len(keep_dis_ls[i])
        )

        mp_counts_ls2.append(mp_counts)
        with open(file_loc, "wb") as f:
            pickle.dump(mp_counts, f)
print("Done.")

Extracting positive metapath pair counts ...


100%|███████████████████████████████████████████████████████████████████████████████████████████████| 53/53 [48:20<00:00, 54.73s/it]

Done.


In [17]:
for i, mp_counts in enumerate(mp_counts_ls2):
    print(
        f'Percent of paths populated: {(mp_counts["pair_count"] != 0).sum() / len(mp_counts):1.2%}'
    )

Percent of paths populated: 69.05%
Percent of paths populated: 74.30%


### Get idealized training subset metapaths

In [18]:
comp_group = [
    "indication_CiD",
    "indication_CiG",
    "indication_CiP",
    "indication_CiPW",
    "treats_CtP",
]

dis_group = [
    "treats_GtD",
    "indication_AiD",
    "indication_CiD",
]

In [19]:
# Ideal Training Subset

neg_frac = 0.1
rs = 1234

keep_comps_ls = [
    set(edges.query('type == "treats_CtD" | type in @comp_group')["start_id"])
    for edges in edges_ls
]
keep_comps_ls = [
    keep_comps
    | set(
        nodes.query('id not in @keep_comps and label == "ChemicalSubstance"').sample(
            frac=neg_frac, random_state=rs
        )["id"]
    )
    for keep_comps in keep_comps_ls
]

keep_dis_ls = [
    set(edges.query('type == "treats_CtD" | type in @dis_group')["end_id"])
    for edges in edges_ls
]
keep_dis_ls = [
    keep_dis
    | set(
        nodes.query('label == "Disease" and id not in @keep_dis').sample(
            frac=neg_frac, random_state=rs + 1
        )["id"]
    )
    for keep_dis in keep_dis_ls
]

for i, keep_comps in enumerate(keep_comps_ls):
    print_mem_info(len(keep_comps), len(keep_dis_ls[i]), len(non_sim_mps_ls[i]))

14,543 Compounds * 6,510 Diseases = 94,674,930 C-D Pairs
94,674,930 C-D Pairs * 8,198 Metapaths = 776,145,076,140 Matrix Values
2,891.4 GB of matrix values
0.353 GB per metapath
14,540 Compounds * 6,506 Diseases = 94,597,240 C-D Pairs
94,597,240 C-D Pairs * 8,198 Metapaths = 775,508,173,520 Matrix Values
2,889.0 GB of matrix values
0.352 GB per metapath


In [20]:
# This can take like 2 hours
print("Extracting training samples ...")

mp_counts_ls3 = list()
for i, mg in enumerate(mg_ls):
    file_loc = os.path.join("./path-based/rephetio/0_data", f"mp_counts2_{i}.pkl")

    if os.path.exists(file_loc):
        with open(file_loc, "rb") as f:
            mp_counts = pickle.load(f)

        mp_counts_ls.append(mp_counts)
    else:
        mp_counts = piecewise_extraction(
            function=mg.extract_metapath_pair_counts,
            to_split="metapaths",
            block_size=500,
            metapaths=[mp.abbrev for mp in mps_ls[i]],
            start_nodes=list(keep_comps_ls[i]),
            end_nodes=list(keep_dis_ls[i]),
            n_jobs=30,
        )
        mp_counts["subset"] = "traning_sample"
        mp_counts["frac"] = mp_counts["pair_count"] / (
            len(keep_comps_ls[i]) * len(keep_dis_ls[i])
        )

        mp_counts_ls3.append(mp_counts)

        with open(file_loc, "wb") as f:
            pickle.dump(mp_counts, f)

print("Done.")

Extracting training samples ...


100%|███████████████████████████████████████████████████████████████████████████████████████████████| 53/53 [50:52<00:00, 57.60s/it]

Done.


In [21]:
for mp_counts in mp_counts_ls3:
    print(
        f"Percent of paths populated: {(mp_counts['pair_count'] != 0).sum() / len(mp_counts):1.2%}"
    )

Percent of paths populated: 69.15%
Percent of paths populated: 74.40%


### Get All training negatives metapath counts

In [22]:
# All Training Negatives
# Get potential training negatives as compounds or diseases with no known treatments
pos_comps_ls = [
    set(edges.query('type == "treats_CtD" | type in @comp_group')["start_id"])
    for edges in edges_ls
]
pos_dis_ls = [
    set(edges.query('type == "treats_CtD" | type in @dis_group')["end_id"])
    for edges in edges_ls
]

neg_comps_ls = [
    set(nodes.query('label == "ChemicalSubstance"')["id"]) - pos_comps_ls[i]
    for i, nodes in enumerate(nodes_ls)
]
neg_dis_ls = [
    set(nodes.query('label == "Disease"')["id"]) - pos_dis_ls[i]
    for i, nodes in enumerate(nodes_ls)
]

for i, neg_comps in enumerate(neg_comps_ls):
    print_mem_info(len(neg_comps), len(neg_dis_ls[i]), len(non_sim_mps_ls[i]))

28,791 Compounds * 8,258 Diseases = 237,756,078 C-D Pairs
237,756,078 C-D Pairs * 8,198 Metapaths = 1,949,124,327,444 Matrix Values
7,261.1 GB of matrix values
0.886 GB per metapath
28,794 Compounds * 8,262 Diseases = 237,896,028 C-D Pairs
237,896,028 C-D Pairs * 8,198 Metapaths = 1,950,271,637,544 Matrix Values
7,265.3 GB of matrix values
0.886 GB per metapath


In [23]:
# This can take like 2 hours
print("Extracting negative pair counts ...")

mp_counts_ls4 = list()
for i, mg in enumerate(mg_ls):
    file_loc = os.path.join("./path-based/rephetio/0_data", f"mp_counts3_{i}.pkl")

    if os.path.exists(file_loc):
        with open(file_loc, "rb") as f:
            mp_counts = pickle.load(f)

        mp_counts_ls.append(mp_counts)

    else:
        mp_counts = piecewise_extraction(
            function=mg.extract_metapath_pair_counts,
            to_split="metapaths",
            block_size=350,
            metapaths=[mp.abbrev for mp in mps_ls[i]],
            start_nodes=list(neg_comps_ls[i]),
            end_nodes=list(neg_dis_ls[i]),
            n_jobs=30,
        )
        mp_counts["subset"] = "negative_pairs"
        mp_counts["frac"] = mp_counts["pair_count"] / (
            len(keep_comps_ls[i]) * len(keep_dis_ls[i])
        )
        mp_counts_ls4.append(mp_counts)

        with open(file_loc, "wb") as f:
            pickle.dump(mp_counts, f)

print("Done.")

Extracting negative pair counts ...


100%|███████████████████████████████████████████████████████████████████████████████████████████████| 76/76 [53:26<00:00, 42.19s/it]

Done.


### Get a snapshot of all dataframes

In [24]:
non_sim_mp_names_ls = [
    [mp.abbrev for mp in non_sim_mps] for non_sim_mps in non_sim_mps_ls
]

all_mp_counts_ls, no_sim_mp_counts_ls, sim_mp_counts_ls = list(), list(), list()
for i, mp_counts in enumerate(mp_counts_ls):
    all_mp_counts = pd.concat(
        [mp_counts, mp_counts_ls2[i], mp_counts_ls3[i], mp_counts_ls4[i]],
        sort=False,
        ignore_index=True,
    )
    # all_mp_counts = pd.concat(all_mp_counts, sort=False, ignore_index=True)
    all_mp_counts["sim_mp"] = all_mp_counts["mp"].apply(
        lambda m: m not in non_sim_mp_names_ls[i]
    )

    no_sim_mp_counts = all_mp_counts.query("sim_mp == False")
    sim_mp_counts = all_mp_counts.query("sim_mp == True")

    all_mp_counts_ls.append(all_mp_counts)
    no_sim_mp_counts_ls.append(no_sim_mp_counts)
    sim_mp_counts_ls.append(sim_mp_counts)

### Export the metapath counts

In [26]:
all_mp_counts_ls[0].to_csv(
    "./path-based/rephetio/0_data/test/all_mp_counts.csv", index=False
)
all_mp_counts_ls[1].to_csv(
    "./path-based/rephetio/0_data/valid/all_mp_counts.csv", index=False
)

## Hyperparameter optimization
* Optimize Degree weighted path count hyperparameter

### import valid data for training hyperparameters

In [3]:
# holdout is valid
edges_no_valid = pd.read_csv("./data/remain_edges_no-valid.csv", dtype=str)
edges_valid = pd.read_csv("./data/holdout_edges_valid.csv")

# holdout is test
edges_no_test = pd.read_csv("./data/remain_edges_no-test.csv", dtype=str)
edges_test = pd.read_csv("./data/holdout_edges_test.csv")
nodes = pd.read_csv("./data/nodes_rep.csv")

# add to a list
nodes_ls = [nodes, nodes]
edges_ls = [edges_no_valid, edges_no_test]

### create the graph formatted as a matrix

In [4]:
# create the matrix formatted graph
mg_ls = list()
mg = MatrixFormattedGraph(
    nodes,
    edges_no_valid,
    "ChemicalSubstance",
    "Disease",
    max_length=4,
    w=0.4,
    n_jobs=30,
)

mg_ls.append(mg)

mg = MatrixFormattedGraph(
    nodes,
    edges_no_test,
    "ChemicalSubstance",
    "Disease",
    max_length=4,
    w=0.4,
    n_jobs=30,
)

mg_ls.append(mg)

Processing node and edge data...
Initializing metagraph...
Generating adjacency matrices...


100%|███████████████████████████████████████████████████████████████████████████████████| 74/74 [00:53<00:00,  1.40it/s]



Determining degrees for each node and metaedge


100%|███████████████████████████████████████████████████████████████████████████████████| 74/74 [00:25<00:00,  2.87it/s]



Weighting matrices by degree with dampening factor 0.4...


...

100%|███████████████████████████████████████████████████████████████████████████████████| 75/75 [00:54<00:00,  1.37it/s]



Determining degrees for each node and metaedge


100%|███████████████████████████████████████████████████████████████████████████████████| 75/75 [00:25<00:00,  2.90it/s]



Weighting matrices by degree with dampening factor 0.4...


100%|███████████████████████████████████████████████████████████████████████████████████| 75/75 [00:00<00:00, 82.50it/s]


### HPO imports

In [5]:
from scipy.sparse import issparse, csc_matrix, csr_matrix

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MaxAbsScaler
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_selection import SelectKBest, chi2, RFE, SelectFromModel
from sklearn.metrics import roc_auc_score, average_precision_score

from xgboost import XGBClassifier

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

from hetnet_ml.extractor import piecewise_extraction
from data_tools.ml import get_model_coefs

### Data Transformer
* An inverse hyperbolic sine transformation is used to transfomr the features

* The transormation is thus:

$$sinh^{-1} \left( \frac{X_{mp}}{\sigma_{mp}} \right)$$

where $X_{mp}$ is the column in the dwpc feature matrix $X$ coreesponding to metapath $mp$ and $\sigma_{mp}$ is the mean of the column.

In [6]:
class MeanScaledArcsinhTransformer(TransformerMixin):

    def fit(self, X, y=None):
        if issparse(X):
            self.initial_mean_ = X.tocoo().tocsc().mean(axis=0).A[0]
        else:
            self.initial_mean_ = X.mean(axis=0)

        # If input was DataFrame, Converts resultant series to ndarray
        try:
            self.initial_mean_ = self.initial_mean_.values
        except:
            pass

        # If inital mean == 0, likely all values were zero
        # this prevents issues later.
        self.initial_mean_[np.where(self.initial_mean_ == 0.0)] = 1

        return self

    def transform(self, X, y=None):
        if issparse(X):
            return np.arcsinh(X.tocoo().tocsc().multiply(self.initial_mean_**-1))
        return np.arcsinh(X / self.initial_mean_)

### Homegrown feature selector

Runs 6 analysis on the traning data to select features.

1. Correlation to the output
2. Chi_squared test
3. Recursive Feature Elimination with a Ridge Regressor
4. Embedded Feature Selection from a Lasso Regressor
5. Embedded Feature Selection from a Randomn Forest Classifier
6. Embedded Feature Selection from a Gradient Boosting Classifier

Each analysis will select `num_feats` best features. The selected features will then by chosen via a voting method with `min_selections` out of the 6 elements required to for a feature to be kept. 


We have also added an option for `always_keep`:  This allows for domain expertise to be factored into the feature selection process.  In our case, we know some metapaths are specifically mechanistic, so we want to include those wherever possible

In [7]:
def cor_selector(X, y, feature_names, num_feats):
    cor_list = []
    # calculate the correlation with y for each feature
    for i in range(X.shape[1]):
        if issparse(X):
            x = X[:, i].A.reshape(len(y))
        else:
            x = X[:, i]
        cor = np.corrcoef(x, y)[0, 1]
        cor_list.append(cor)
    # replace NaN with 0
    cor_list = [0 if np.isnan(i) else i for i in cor_list]
    # feature name
    cor_feature = np.array(feature_names)[
        np.argsort(np.abs(cor_list))[-num_feats:].tolist()
    ].tolist()
    # feature selection? 0 for not select, 1 for select
    return [True if i in cor_feature else False for i in feature_names]


def chi2_selector(X, y, num_feats):
    this_selector = SelectKBest(chi2, k=num_feats)
    this_selector.fit(X, y)
    return this_selector.get_support()


def rfe_selector(X, y, num_feats, random_state=None):
    this_selector = RFE(
        estimator=LogisticRegression(
            C=0.1, solver="liblinear", random_state=random_state
        ),
        n_features_to_select=num_feats,
        step=0.2,
        verbose=5,
    )
    this_selector.fit(X, y)
    return this_selector.get_support()


def embeded_lr_selector(X, y, num_feats, random_state=None):
    this_selector = SelectFromModel(
        LogisticRegression(penalty="l1", solver="liblinear", random_state=random_state),
        max_features=num_feats,
    )
    this_selector.fit(X, y)

    return this_selector.get_support()


def embeded_rf_selector(X, y, num_feats, n_jobs, random_state=None):
    rfc = RandomForestClassifier(
        n_estimators=100, max_depth=50, n_jobs=n_jobs, random_state=random_state
    )
    this_selector = SelectFromModel(rfc, max_features=num_feats)
    this_selector.fit(X, y)
    return this_selector.get_support()


def embeded_xgb_selector(X, y, num_feats, n_jobs=1, random_state=None):
    # XGBoost takes 0 as default random state
    if random_state is None:
        random_state = 0
    # Paramaters optimized for speed, rather than accuracy (as we have 5 other estimators also providing votes)
    xgbc = XGBClassifier(
        max_depth=5,
        n_estimators=200,
        learning_rate=0.16,
        min_child_weight=1,
        colsample_bytree=0.8,
        n_jobs=n_jobs,
        random_state=random_state,
    )
    this_selector = SelectFromModel(xgbc, max_features=num_feats)
    this_selector.fit(X, y)
    return this_selector.get_support()


class FeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(
        self,
        num_features=100,
        min_selections=4,
        n_jobs=1,
        feature_names=None,
        always_keep=None,
        random_state=None,
    ):
        self.num_features = num_features
        self.min_selections = min_selections
        self.n_jobs = n_jobs
        self.feature_names = feature_names
        self.always_keep = always_keep
        self.random_state = random_state

    def fit(self, X, y=None):

        X_norm = MaxAbsScaler().fit_transform(X)
        if issparse(X):
            if type(X) != csc_matrix:
                X = X.tocsc()
            X_norm = X_norm.tocsc()

        print("Running Cor")
        cor_support = cor_selector(X, y, self.feature_names, self.num_features)
        print("Running Chi2")
        chi_support = chi2_selector(X_norm, y, self.num_features)
        print("Running RFE")
        rfe_support = rfe_selector(X_norm, y, self.num_features, self.random_state)
        print("Running LR")
        embeded_lr_support = embeded_lr_selector(
            X_norm, y, self.num_features, self.random_state
        )
        print("Running RF")
        embeded_rf_support = embeded_rf_selector(
            X, y, self.num_features, n_jobs=self.n_jobs, random_state=self.random_state
        )
        print("Running XG")
        embeded_xgb_support = embeded_xgb_selector(
            X, y, self.num_features, n_jobs=self.n_jobs, random_state=self.random_state
        )

        feature_selection_df = pd.DataFrame(
            {
                "feature": self.feature_names,
                "pearson": cor_support,
                "chi_2": chi_support,
                "rfe": rfe_support,
                "logistics": embeded_lr_support,
                "random_forest": embeded_rf_support,
                "xgboost": embeded_xgb_support,
            }
        )

        feature_selection_df["total"] = np.sum(feature_selection_df.iloc[:, 1:], axis=1)
        self.feature_selection_df_ = feature_selection_df

        keep_features = feature_selection_df.query(
            "total >= {}".format(self.min_selections)
        )["feature"].tolist()

        # Keep the features that we always want (e.g. domain expertise)
        if self.always_keep is not None:
            keep_features.extend(self.always_keep)

        self.keep_features_ = [f for f in self.feature_names if f in keep_features]

        return self

    def transform(self, X, y=None):

        if issparse(X) and type(X) != csc_matrix:
            X = X.tocsc()
        return X[
            :, [i for i, f in enumerate(self.feature_names) if f in self.keep_features_]
        ]

### Split Data for HPO

In [8]:
# get the types that are relevant
comp_group = [
    "indication_CiD",
    "indication_CiG",
    "indication_CiP",
    "indication_CiPW",
    "treats_CtP",
    "treats_CtD",
]

dis_group = [
    "treats_GtD",
    "treats_CtD",
    "indication_AiD",
    "indication_CiD",
]

#### further split the dataset missing the valid/test indications
* If you recall the dataset was split by 80/10/10 by train/valid/test
* the 80% is further separated by 85/15% for HPO
* finally the evaluation will occur on the remaining 10% for valid/test.

In [9]:
train_frac = 0.15
rs = 20200123

treat_comps_ls = [
    set(edges.query("type in @comp_group")["start_id"]) for edges in edges_ls
]
# Sample the negatives and subsample
keep_comps_ls = [
    set(
        nodes.query(
            'id not in @treat_comps_ls[@i] and label == "ChemicalSubstance"'
        ).sample(frac=train_frac * 0.01, random_state=rs)["id"]
    )
    for i, nodes in enumerate(nodes_ls)
]
# Then subsample the positives
keep_comps_ls = [
    keep_comps_ls[i]
    | set(
        nodes.query("id in @treat_comps_ls[@i]").sample(
            frac=train_frac, random_state=rs + 1
        )["id"]
    )
    for i, nodes in enumerate(nodes_ls)
]

treat_dis_ls = [set(edges.query("type in @dis_group")["end_id"]) for edges in edges_ls]
# Sample the negatives and subsample cv
keep_dis_ls = [
    set(
        nodes.query('label == "Disease" and id not in @treat_dis_ls[@i]').sample(
            frac=train_frac * 0.01, random_state=rs + 2
        )["id"]
    )
    for i, nodes in enumerate(nodes_ls)
]
# Take the diseases Treated by these compounds
keep_dis_ls = [
    keep_dis_ls[i]
    | set(
        edges.query("type in @dis_group and start_id in @keep_comps_ls[@i]")["end_id"]
    )
    for i, edges in enumerate(edges_ls)
]

In [10]:
# MP counts will be the same with or without weights, so just use the original
save_folder = "./path-based/rephetio/0_data"
all_mp_counts_ls = [
    pd.read_csv(os.path.join(save_folder, "valid", "all_mp_counts.csv")),
    pd.read_csv(os.path.join(save_folder, "test", "all_mp_counts.csv")),
]

all_mp_counts_ls[0].head(2)

,mp,pair_count,subset,frac,sim_mp
0,CpoAawD,5854758,all_pairs,2.017724e+06,False
1,CpoAiD,0,all_pairs,0.000000e+00,False


In [11]:
# estimate the memory required to extract all compound-disease pairs
float_size = 32  # bits
bits_per_gb = 8589934592


def print_mem_info(n_comp, n_dis, n_mps):
    print(f"{n_comp:,} Compounds * {n_dis:,} Diseases = {n_comp * n_dis:,} C-D Pairs")
    print(
        f"{n_comp * n_dis:,} C-D Pairs * {n_mps:,} Metapaths = {n_comp * n_dis * n_mps:,} Matrix Values"
    )
    print(
        f"{n_comp * n_dis * n_mps * float_size / (bits_per_gb):1,.1f} GB of matrix values"
    )
    print(f"{n_comp * n_dis * float_size / (bits_per_gb):1,.3f} GB per metapath")

In [12]:
non_sim_names_ls = [
    all_mp_counts.query("sim_mp == False")["mp"].unique().tolist()
    for all_mp_counts in all_mp_counts_ls
]
mp_qr_ls = [
    all_mp_counts.query(
        'subset == "all_pairs" and mp in @non_sim_names_ls[@i] and pair_count > 0'
    )
    for i, all_mp_counts in enumerate(all_mp_counts_ls)
]
good_mps_ls = [mp_qr["mp"].tolist() for mp_qr in mp_qr_ls]

save_folder_ls = [os.path.join(save_folder, "valid"), os.path.join(save_folder, "test")]
for i, keep_comps in enumerate(keep_comps_ls):
    print(f"{save_folder_ls[i]}")
    print_mem_info(len(keep_comps), len(keep_dis_ls[i]), len(good_mps_ls[i]))

./path-based/rephetio/0_data/valid
1,792 Compounds * 3,123 Diseases = 5,596,416 C-D Pairs
5,596,416 C-D Pairs * 6,392 Metapaths = 35,772,291,072 Matrix Values
133.3 GB of matrix values
0.021 GB per metapath
./path-based/rephetio/0_data/test
1,793 Compounds * 3,195 Diseases = 5,728,635 C-D Pairs
5,728,635 C-D Pairs * 5,814 Metapaths = 33,306,283,890 Matrix Values
124.1 GB of matrix values
0.021 GB per metapath


In [13]:
for i, edges in enumerate(edges_ls):
    print(
        f"""{len(edges.query("start_id in @keep_comps_ls[@i] and (type == 'indication_CiD' | type =='treats_CtD')")):,} Positive training examples in subset"""
    )

10,948 Positive training examples in subset
11,167 Positive training examples in subset


### Bring in known metapaths from DrugMechDB

In [14]:
mech_mps = pd.read_csv("./path-based/rephetio/0_data/manual/mech_mps.txt", header=None)[
    0
].values
dmdb_feat_ls = [
    set(
        all_mp_counts.query(
            'mp in @mech_mps and subset == "all_pairs" and pair_count > 0 and sim_mp == False'
        )["mp"]
    )
    for all_mp_counts in all_mp_counts_ls
]

for dmdb_feat in dmdb_feat_ls:
    len(dmdb_feat)

### Extract the features
Use the pair counts to sort metapath extractions as a naive load balancer

In [15]:
def sort_mps_for_pw_extraction(n_big_calcs, big_per_block, mp_list, frac_info):

    big_mp = (
        frac_info.sort_values("frac", ascending=False).head(n_big_calcs)["mp"].tolist()
    )
    other_mp = list(set(mp_list) - set(big_mp))

    block_size = len(other_mp) // (len(big_mp) // big_per_block)
    n_blocks = (len(big_mp) + len(other_mp)) // block_size

    out = []
    for i in range(n_blocks):
        for j in range(big_per_block):
            idx = i * big_per_block + j
            out.append(big_mp[idx])
        out += other_mp[i * block_size : (i + 1) * block_size]

    out += list(set(other_mp) - set(out))

    return out, block_size

In [16]:
# 30, 5 finishes < 30 min. <- OVERFLOW
# 40, 8 finishes 18min 46s <- OVERFLOW

to_xtract_ls, block_size_ls = list(), list()
for i, good_mps in enumerate(good_mps_ls):
    to_xtract, block_size = sort_mps_for_pw_extraction(100, 5, good_mps, mp_qr_ls[i])
    to_xtract_ls.append(to_xtract)
    block_size_ls.append(block_size)

In [17]:
len(to_xtract_ls[0]), block_size_ls[0]
len(to_xtract_ls[1]), block_size_ls[1]

(5814, 285)

In [19]:
# Extract the metapaths to do some prep work
pairs_ls, feats_ls, test_dwpc_ls = list(), list(), list()
for save_folder in save_folder_ls:

    # get locations of each file
    pairs_loc = os.path.join(save_folder, "pairs.pkl")
    feats_loc = os.path.join(save_folder, "feats.pkl")
    test_dwpc_loc = os.path.join(save_folder, "test_dwpc.pkl")

    # import files
    if (
        os.path.exists(pairs_loc)
        and os.path.exists(feats_loc)
        and os.path.exists(test_dwpc_loc)
    ):
        with open(pairs_loc, "rb") as f:
            pairs = pickle.load(f)
        with open(feats_loc, "rb") as f:
            feats = pickle.load(f)
        with open(test_dwpc_loc, "rb") as f:
            test_dwpc = pickle.load(f)
        pairs_ls.append(pairs)
        feats_ls.append(feats)
        test_dwpc_ls.append(test_dwpc)

    else:
        for i, mg in enumerate(mg_ls):
            (pairs, feats), test_dwpc = piecewise_extraction(
                function=mg.extract_dwpc,
                to_split="metapaths",
                block_size=block_size_ls[i],
                axis=1,
                metapaths=to_xtract_ls[i],
                start_nodes=list(keep_comps_ls[i]),
                end_nodes=list(keep_dis_ls[i]),
                return_sparse=True,
                sparse_df=False,
                n_jobs=4,
            )

            pairs_ls.append(pairs)
            feats_ls.append(feats)
            test_dwpc_ls.append(test_dwpc)

            with open(pairs_loc, "wb") as f:
                pickle.dump(pairs, f)
            with open(feats_loc, "wb") as f:
                pickle.dump(feats, f)
            with open(test_dwpc_loc, "wb") as f:
                pickle.dump(test_dwpc, f)

In [20]:
pos_tups_ls = [
    edges.query('type == "indication_CiD" | type == "treats_CtD"')[
        ["start_id", "end_id"]
    ]
    .apply(tuple, axis=1)
    .tolist()
    for edges in edges_ls
]

pos_tups_ls[0][:2]

[('CHEBI:100147', 'DOID:0050400'), ('CHEBI:100147', 'DOID:13148')]

In [21]:
pos_tups_ls = [set(pos_tups) for pos_tups in pos_tups_ls]

In [22]:
y_ls = list()
for i, pairs in enumerate(pairs_ls):
    y = list()

    for row in tqdm(pairs.itertuples(), total=len(pairs)):
        if set([(row.chemicalsubstance_id, row.disease_id)]) & pos_tups_ls[i]:
            y.append(1)
        else:
            y.append(0)
    y = np.array(y)

    pairs["status"] = y

    print(
        f"Total Chemical to Disease Pairs: {len(pairs)}\nlength: {len(y)}\ntotal: {sum(y)}"
    )

    pairs_ls[i] = pairs
    y_ls.append(y)

  0%|                                                                                       | 0/5596416 [00:00<?, ?it/s]

100%|████████████████████████████████████████████████████████████████████| 5596416/5596416 [00:04<00:00, 1122911.97it/s]


Total Chemical to Disease Pairs: 5596416
length: 5596416
total: 10939


100%|████████████████████████████████████████████████████████████████████| 5596416/5596416 [00:04<00:00, 1127626.55it/s]


Total Chemical to Disease Pairs: 5596416
length: 5596416
total: 10924


### Subsample the potential training examples

* Our class is so imbalanced, to get a sizeable number of positive training examples, we end up with many orders of magnitude more negative examples.  Many of those examples will have no connections from the compound to the disease of interest, providing a zero row in the matrix. We will not waste time training on those values, and instead focus on the ones that distinguish the positive from the negative training examples.

* As this is just hyperparameter tunings, to speed things up, we will also limit the positive examples to a small portion of the negative examples, 100x (2 orders of magnitude) larger than the number of postitives. 

In [23]:
# Get the rows that are Non-zero
nz_index_ls = [
    pairs[test_dwpc_ls[i].getnnz(1) > 0].index for i, pairs in enumerate(pairs_ls)
]

# have the number of postivies to get 100x this for the negatives.
n_pos_ls = [pairs["status"].sum() for pairs in pairs_ls]

# Sample the nonzero negative examples at a rate of 100x the positive samples
neg_index_ls = [
    pairs.loc[nz_index_ls[i]]
    .query("status == 0")
    .sample(n=100 * n_pos_ls[i], random_state=rs + 10)
    .sort_index()
    .index
    for i, pairs in enumerate(pairs_ls)
]

# and of course take the training postivies
pos_index_ls = [pairs.query("status == 1").index for pairs in pairs_ls]

# Union the two
train_index_ls = [
    pos_index.union(neg_index_ls[i]) for i, pos_index in enumerate(pos_index_ls)
]

In [24]:
feats_ls = [np.array(feats) for feats in feats_ls]
nz_feats_ls = [feats[test_dwpc_ls[i].getnnz(0) > 0] for i, feats in enumerate(feats_ls)]
feat_index_ls = [test_dwpc.getnnz(0) > 0 for test_dwpc in test_dwpc_ls]

In [25]:
# Get our compounds as ndarrays for easy indexing with sklearns StratifiedKFold
keep_comps_ls = [np.array(list(keep_comps)) for keep_comps in keep_comps_ls]
# Need to know how to properly stratify the split
is_treat_comp_ls = [
    np.array([1 if c in treat_comps else 0 for c in keep_comps_ls[i]])
    for i, treat_comps in enumerate(treat_comps_ls)
]

### Select the features
This is a time consuming and expensive step.
We will do it once with the initial DWPC for parameter tuning and perform it again after parameter selection

In [42]:
msat = MeanScaledArcsinhTransformer()
trans_dwpc_ls = [
    msat.fit_transform(test_dwpc[train_index_ls[i]][:, feat_index_ls[i]])
    for i, test_dwpc in enumerate(test_dwpc_ls)
]

In [43]:
for save_folder in save_folder_ls:
    tmp_dir = os.path.join(save_folder, "tmp")
    try:
        os.mkdir(tmp_dir)
    except:
        pass

In [ ]:
keep_features_ls = list()

for i, save_folder in enumerate(save_folder_ls):
    tmp_dir = os.path.join(save_folder, "tmp")
    if os.path.exists(os.path.join(tmp_dir, "test_feats.txt")):
        keep_features = pd.read_csv(os.path.join(tmp_dir, "test_feats.txt"))[
            "0"
        ].tolist()
    else:
        fsel = FeatureSelector(
            num_features=500,
            min_selections=4,
            n_jobs=30,
            feature_names=nz_feats_ls[i].tolist(),
            always_keep=dmdb_feat_ls[i],
            random_state=rs + 5,
            # save=save_folder,
        )
        sel_dwpc = fsel.fit_transform(trans_dwpc_ls[i], y[train_index_ls[i]])
        keep_features = fsel.keep_features_
        pd.Series(keep_features).to_csv(
            os.path.join(tmp_dir, "test_feats.txt"), index=False
        )
        fsel.feature_selection_df_.to_csv(
            os.path.join(tmp_dir, "test_fs_df.csv"), index=False
        )
    keep_features_ls.append(keep_features)

### Prep HPO

In [117]:
all_dwpc = dict()


def get_dwpc(w, i):
    global all_dwpc

    # only extract if not previously extracted
    if w not in all_dwpc.keys():
        mg.update_w(w)
        (pairs, feats), dwpc = mg.extract_dwpc(
            metapaths=keep_features,
            start_nodes=list(keep_comps_ls[i]),
            end_nodes=list(keep_dis_ls[i]),
            return_sparse=True,
            sparse_df=False,
            n_jobs=4,
        )
        # next step is split, so need sparse rows
        dwpc = dwpc.tocoo().tocsr()
        # all_dwpc[w] = dwpc
        return dwpc
    else:
        return all_dwpc[w]


def neg_log_scal(val):
    """Scale values logrithmicly. Larger input yields smaller result"""
    return 1 - np.log1p(val)


def pos_log_scal(val):
    """Scale values logrithmicly. Larger input yields larger result"""
    return np.log1p(val)


def auroc_mean_loss(auroc_mean, strength=2, scal_factor=0.5):
    mean_roc_shift = (np.abs(auroc_mean - 0.5)) / scal_factor
    baseline_roc = neg_log_scal(1)
    return (neg_log_scal(mean_roc_shift) - baseline_roc) * strength


def avg_pre_mean_loss(avg_pre_mean, strength=2.5, scal_factor=0.15):
    avg_pre_mean_scal = avg_pre_mean / scal_factor
    baseline_prc = neg_log_scal(1)
    return (neg_log_scal(avg_pre_mean_scal) - baseline_prc) * strength


def auroc_std_loss(auroc_std, strength=2):
    return pos_log_scal(auroc_std) * strength


def avg_pre_std_loss(avg_pre_std, strength=2):
    return pos_log_scal(avg_pre_std) * strength


def sigmoid(x):
    return 1 / (1 + np.exp(-x))


def calc_loss(auroc_mean, avg_pre_mean, auroc_std, avg_pre_std):
    mean_roc_loss = auroc_mean_loss(auroc_mean)
    mean_pre_loss = avg_pre_mean_loss(avg_pre_mean)
    std_roc_loss = auroc_std_loss(auroc_std)
    std_pre_loss = avg_pre_std_loss(avg_pre_std)

    return sigmoid(mean_roc_loss + mean_pre_loss + std_roc_loss + std_pre_loss) - 1

In [118]:
def hyperopt(param_space, y, num_eval, i):
    def objective_function(params):
        dwpc_params = {
            k.split("__")[1]: v for k, v in params.items() if k.split("__")[0] == "dwpc"
        }
        enet_params = {
            k.split("__")[1]: v for k, v in params.items() if k.split("__")[0] == "enet"
        }

        this_w = dwpc_params["w"]

        # Set up the post feature extraction pipeline
        post_extraction_pipeline = Pipeline(
            [
                ("transformer", MeanScaledArcsinhTransformer()),
                ("maxabs_scale", MaxAbsScaler()),
                (
                    "e_net",
                    LogisticRegression(
                        penalty="elasticnet",
                        solver="saga",
                        max_iter=100,
                        **enet_params,
                        random_state=rs + 6,
                    ),
                ),
            ],
            verbose=True,
        )

        ## Get the dwpc information for the current pairs
        dwpc = get_dwpc(this_w, i)
        this_dwpc = dwpc[train_index_ls[i]]

        cv = cross_validate(
            post_extraction_pipeline,
            this_dwpc,
            y,
            cv=5,
            scoring=["average_precision", "roc_auc"],
            return_estimator=True,
        )

        # Write out scores for each run
        with open(
            os.path.join(
                tmp_dir,
                f"scores_w_{this_w:1.4f}_C_{enet_params['C']:1.5f}_l1_{enet_params['l1_ratio']:1.4f}.txt",
            ),
            "w",
        ) as f_out:

            f_out.write(", ".join([str(s) for s in cv["test_average_precision"]]))
            f_out.write("\n")
            f_out.write(", ".join([str(s) for s in cv["test_roc_auc"]]))
            f_out.write("\n")

        auroc_mean = cv["test_roc_auc"].mean()
        avg_pre_mean = cv["test_average_precision"].mean()
        auroc_std = cv["test_roc_auc"].std()
        avg_pre_std = cv["test_average_precision"].std()

        score = calc_loss(auroc_mean, avg_pre_mean, auroc_std, avg_pre_std)

        print("Mean AUROC: {:1.4f}".format(auroc_mean))
        print("Mean Avg Pre: {:1.4f}".format(avg_pre_mean))
        print("STD AUROC: {:1.4f}".format(auroc_std))
        print("STD Avg Pre: {:1.4f}".format(avg_pre_std))
        print("Loss: {:1.4f}".format(score))

        return {"loss": score, "status": STATUS_OK}

    start = time()

    trials = Trials()
    # load the pickle file if you have a version saved.
    pkl_file = os.path.join(tmp_dir, "trials.pkl")
    if os.path.exists(pkl_file):
        with open(pkl_file, "rb") as f:
            trials = pickle.load(f)

    best_param = fmin(
        objective_function,
        param_space,
        algo=tpe.suggest,
        max_evals=num_eval,
        trials=trials,
        rstate=np.random.default_rng(seed=None),  # Removed fixed starting seed.
    )

    print(time() - start)
    return trials, best_param

In [119]:
# Previous best before adding new keep_features:
# '1l_ratio': 0.10455936193818496, 'C': 0.000556880900960339, 'w': 0.2640929485381926
param_hyperopt = {
    "dwpc__w": hp.uniform("w", 0.01, 1),
    "enet__C": hp.loguniform("C", np.log(0.001), np.log(1)),
    "enet__l1_ratio": hp.uniform("l1_ratio", 0.01, 0.99),
}

In [ ]:
# about 1100 minutes
best_param_ls = list()

# check if you already exported your trials.pkl file. if you have already, skip the time consuming step
for i in save_folder_ls:
    if os.path.exists(os.path.join(i, "trials.pkl")):
        with open(os.path.join(i, "trials.pkl"), "rb") as f:
            trial = pickle.load(f)

        best_param_ls.append(trial.best_trial["misc"]["vals"])

if len(best_param_ls) < 5:
    # this loop runs 50 iterations of hyperparameter optimization for each slice of cross validation
    for i, y in enumerate(y_ls):
        # open a temporary directory in the directory of interest to store our files in
        tmp_dir = os.path.join(save_folder_ls[i], "tmp")

        # incase we run out of memory, sets number of trials
        num_eval = 50
        num_evaluated = (
            sum(
                [
                    i.startswith("scores") and i.endswith(".txt")
                    for i in os.listdir(tmp_dir)
                ]
            )
            + 1
        )  # needs plus 1 otherwise will start at 0
        pkl_file = os.path.join(save_folder_ls[i], "trials.pkl")
        best_param = None
        # crappy for loop to only do 1 hpo at a time since we need to tamp down on memory limitations
        while num_evaluated <= num_eval:

            trials, best_param = hyperopt(
                param_space=param_hyperopt,
                y=y[train_index_ls[i]],
                num_eval=num_evaluated,
                i=i,
            )

            # after every iteration, dump the results
            with open(pkl_file, "wb") as f:
                pickle.dump(trials, f)

            num_evaluated = (
                sum(
                    [
                        file.startswith("scores") and file.endswith(".txt")
                        for file in os.listdir(tmp_dir)
                    ]
                )
                + 1
            )

        best_param_ls.append(best_param)
        with open(os.path.join(save_folder_ls[i], "best_param.pkl"), "wb") as f:
            pickle.dump(best_param, f)

In [122]:
best_param_ls = list()

for i in save_folder_ls:
    with open(os.path.join(i, "trials.pkl"), "rb") as f:
        trial = pickle.load(f)

    best_param_ls.append(trial.best_trial["misc"]["vals"])

# process best_parameters
best_param_ls = [dict(zip(i.keys(), [j[0] for j in i.values()])) for i in best_param_ls]

# return results
for i, v in enumerate(save_folder_ls):
    print(v)
    print(best_param_ls[i])

./path-based/rephetio/0_data/valid
{'C': 0.07073875743215166, 'l1_ratio': 0.014611019497691002, 'w': 0.6649765567763943}
./path-based/rephetio/0_data/test
{'C': 0.009084068877998714, 'l1_ratio': 0.12345440738916719, 'w': 0.9135211198916415}


### Use selected parameters to do feature selection

In [123]:
enet_params_ls = [
    {k: v for k, v in best_param.items() if k != "w"} for best_param in best_param_ls
]
# enet_params = {k: v for k, v in best_param.items() if k != "w"}

post_extraction_pipeline_ls = list()

for i, v in enumerate(best_param_ls):  # just need a list to iterate through
    post_extraction_pipeline = Pipeline(
        [
            ("transformer", MeanScaledArcsinhTransformer()),
            (
                "feature_selection",
                FeatureSelector(
                    num_features=500,
                    min_selections=4,
                    n_jobs=30,
                    feature_names=to_xtract_ls[i],
                    always_keep=dmdb_feat_ls[i],
                    random_state=4,
                ),
            ),
        ],
        verbose=True,
    )

    post_extraction_pipeline_ls.append(post_extraction_pipeline)

In [124]:
dwpc_ls = []
for i, mg in enumerate(mg_ls):
    mg.update_w(best_param_ls[i]["w"])
    (a, b), dwpc = piecewise_extraction(
        function=mg.extract_dwpc,
        to_split="metapaths",
        block_size=block_size,
        axis=1,
        metapaths=to_xtract_ls[i],
        start_nodes=list(keep_comps_ls[i]),
        end_nodes=list(keep_dis_ls[i]),
        return_sparse=True,
        sparse_df=False,
        n_jobs=30,
    )
    dwpc_ls.append(dwpc)
    mg_ls[i] = mg

Changing w from 0.4 to 0.6649765567763943. Please wait...


100%|███████████████████████████████████████████| 23/23 [37:28<00:00, 97.78s/it]


Changing w from 0.6412668503071621 to 0.9135211198916415. Please wait...


100%|██████████████████████████████████████████| 21/21 [36:47<00:00, 105.14s/it]


In [ ]:
post_extraction_pipeline_ls2 = list()
for i, post_extraction_pipeline in enumerate(post_extraction_pipeline_ls):
    j = post_extraction_pipeline.fit(
        dwpc_ls[i][train_index_ls[i]], y_ls[i][train_index_ls[i]]
    )
    post_extraction_pipeline_ls2.append(j)

In [126]:
pe_feats_ls = [i[1].keep_features_ for i in post_extraction_pipeline_ls]

In [127]:
for i, pe_feats in enumerate(pe_feats_ls):
    print(f"pe_feats: {len(pe_feats)}\nkeep_features: {len(keep_features_ls[i])}")
    print(f"Difference: {len(set(keep_features_ls[i])-set(pe_feats))}")  # 39 before

pe_feats: 159
keep_features: 163
Difference: 57
pe_feats: 251
keep_features: 158
Difference: 129


In [128]:
for i, post_extraction_pipeline in enumerate(post_extraction_pipeline_ls):

    # save feature selection dataframe results
    feat_df = post_extraction_pipeline[1].feature_selection_df_
    feat_df.sort_values("total", ascending=False).to_csv(
        os.path.join(save_folder_ls[i], "feature_selection_df.csv"), index=False
    )

    # pickle selected features
    with open(os.path.join(save_folder_ls[i], "feature_selector.pkl"), "wb") as f:
        pickle.dump(post_extraction_pipeline[1], f)

    # save kept features
    pd.Series(post_extraction_pipeline[1].keep_features_).to_csv(
        os.path.join(save_folder_ls[i], "kept_features.txt"), index=False
    )

In [129]:
enet_params_ls = [
    {k: v for k, v in best_param.items() if k != "w"} for best_param in best_param_ls
]

post_extraction_pipeline_ls = list()

for i, enet_params in enumerate(enet_params_ls):
    # Set up the post feature extraction pipeline
    post_extraction_pipeline = Pipeline(
        [
            ("transformer", MeanScaledArcsinhTransformer()),
            ("maxabs_scale", MaxAbsScaler()),
            (
                "e_net",
                LogisticRegression(
                    penalty="elasticnet",
                    solver="saga",
                    max_iter=100,
                    **enet_params,
                    random_state=rs + 6
                ),
            ),
        ],
        verbose=True,
    )
    post_extraction_pipeline_ls.append(post_extraction_pipeline)

dwpc_ls = list()
for i, mg in enumerate(mg_ls):
    ## Get the dwpc information for the current pairs
    (stuff), dwpc = mg.extract_dwpc(
        metapaths=pe_feats_ls[i],
        start_nodes=list(keep_comps_ls[i]),
        end_nodes=list(keep_dis_ls[i]),
        return_sparse=True,
        sparse_df=False,
        n_jobs=4,
    )
    dwpc_ls.append(dwpc)

Preparing function arguments...
Calculating DWPCs...


100%|█████████████████████████████████████████| 159/159 [03:16<00:00,  1.23s/it]



Reshaping Result Matrices...


100%|█████████████████████████████████████████| 159/159 [00:06<00:00, 24.95it/s]


Stacking columns...
Preparing function arguments...
Calculating DWPCs...


100%|█████████████████████████████████████████| 251/251 [02:27<00:00,  1.71it/s]



Reshaping Result Matrices...


100%|█████████████████████████████████████████| 251/251 [00:13<00:00, 18.44it/s]


Stacking columns...


In [130]:
this_dwpc_ls = [dwpc[train_index_ls[i]] for i, dwpc in enumerate(dwpc_ls)]
this_y_ls = [y[train_index_ls[i]] for i, y in enumerate(y_ls)]

In [ ]:
cv_ls, auroc_mean_ls, avg_pre_mean_ls, auroc_std_ls, avg_pre_std_ls, score_ls = (
    list(),
    list(),
    list(),
    list(),
    list(),
    list(),
)

for i, post_extraction_pipeline in enumerate(post_extraction_pipeline_ls):
    cv = cross_validate(
        post_extraction_pipeline,
        this_dwpc_ls[i],
        this_y_ls[i],
        cv=5,
        scoring=["average_precision", "roc_auc"],
        return_estimator=True,
    )

    auroc_mean = cv["test_roc_auc"].mean()
    avg_pre_mean = cv["test_average_precision"].mean()
    auroc_std = cv["test_roc_auc"].std()
    avg_pre_std = cv["test_average_precision"].std()

    score = calc_loss(auroc_mean, avg_pre_mean, auroc_std, avg_pre_std)

    cv_ls.append(cv)
    auroc_mean_ls.append(auroc_mean)
    avg_pre_mean_ls.append(avg_pre_mean)
    auroc_std_ls.append(auroc_std)
    avg_pre_std_ls.append(avg_pre_std)
    score_ls.append(score)

In [132]:
for i, cv in enumerate(cv_ls):
    print(("AUROC Scores: " + 5 * "{:1.4f}, ").format(*cv["test_roc_auc"].tolist()))
    print("Mean AUROC: {:1.4f}".format(auroc_mean_ls[i]))
    print(
        ("Avg Pre. Scores: " + 5 * "{:1.4f}, ").format(
            *cv["test_average_precision"].tolist()
        )
    )
    print("Mean Avg Pre: {:1.4f}".format(avg_pre_mean_ls[i]))
    print(f"STD AUROC: {auroc_std_ls[i]:1.4f}")
    print(f"STD Avg Pre: {avg_pre_std_ls[i]:1.4f}")
    print(f"Loss: {score_ls[i]:1.4f}")

AUROC Scores: 0.7289, 0.7823, 0.7473, 0.7942, 0.6758, 
Mean AUROC: 0.7457
Avg Pre. Scores: 0.0902, 0.1018, 0.0940, 0.2040, 0.0738, 
Mean Avg Pre: 0.1127
STD AUROC: 0.0421
STD Avg Pre: 0.0465
Loss: -0.2513
AUROC Scores: 0.5384, 0.4580, 0.6528, 0.5433, 0.5645, 
Mean AUROC: 0.5514
Avg Pre. Scores: 0.0116, 0.0091, 0.0295, 0.0380, 0.0121, 
Mean Avg Pre: 0.0201
STD AUROC: 0.0623
STD Avg Pre: 0.0115
Loss: -0.0599


## Evaluate model

### some important variables for building the model

In [136]:
mg_ls[1]  # test graph without the test set
edges_test  # test set
best_param_ls[1]  # best parameters
pos_tups_ls[1]  # positive samples

# ids that are referenced in the edges_no_test
ref_c = nodes_ls[1].dropna()["id"].tolist()

# extract the selected features
fs = pickle.load(open(os.path.join(save_folder_ls[1], "feature_selector.pkl"), "rb"))

# metapaths to extract
mp_to_extract = list(set(fs.keep_features_).union(set(dmdb_feat_ls[1])))

# elastic net parameters
enet_params_ls[1]

### Initialize logistic regression with elastic net

In [146]:
post_extraction_pipeline = Pipeline(
    [
        ("transformer", MeanScaledArcsinhTransformer()),
        ("maxabs_scale", MaxAbsScaler()),
        (
            "e_net",
            LogisticRegression(
                penalty="elasticnet",
                solver="saga",
                max_iter=500,
                **enet_params_ls[1],
                random_state=rs + 6
            ),
        ),
    ],
    verbose=True,
)

### train initialized model

In [159]:
# Check if we've already processed the files, if so, skip.
if os.path.exists(os.path.join(save_folder_ls[1], "coef_test.csv")):
    next

else:
    (pairs, out_mp), dwpc = mg_ls[1].extract_dwpc(
        metapaths=mp_to_extract,
        start_nodes=keep_comps_ls[1],
        end_nodes=np.array(list(keep_dis_ls[1])),
        return_sparse=True,
        sparse_df=False,
        verbose=True,
        n_jobs=30,
    )
    # Split the Training and Testing
    print("Shaping Matricies...")
    dwpc = dwpc.tocoo().tocsr()

    print("Getting Target Values")
    y = []
    for row in tqdm(pairs.itertuples(), total=len(pairs)):
        # pos_tups defined several cells above
        if set([(row.chemicalsubstance_id, row.disease_id)]) & pos_tups_ls[1]:
            y.append(1)
        else:
            y.append(0)

    y = np.array(y)
    pairs["status"] = y

    # subset so that we're training only with nonzero dwpc rows
    # Get the training examples that have metapaths
    nz_index = pairs[dwpc.getnnz(1) > 0].index
    # have the number of postivies to get 100x this for the negatives.
    n_pos = pairs["status"].sum()
    # Sample the nonzero negative examples at a rate of 100x the positive samples
    neg_index = (
        pairs.loc[nz_index]
        .query("status == 0")
        .sample(n=100 * n_pos, random_state=rs + 10)
        .sort_index()
        .index
    )
    # and of course take the training postivies
    pos_index = pairs.query("status == 1").index
    # Union the two
    train_index = pos_index.union(neg_index)

    # Fit the model and get results
    print("Training Model")
    post_extraction_pipeline.fit(dwpc[train_index, :], y[train_index])

    print("Getting Model Coef")
    coef = get_model_coefs(
        post_extraction_pipeline[-1], dwpc, mp_to_extract
    ).sort_values("coef", ascending=False)

    # Save model
    coef.to_csv(os.path.join(save_folder_ls[1], "coef_test.csv"), index=False)
    pickle.dump(
        post_extraction_pipeline,
        open(os.path.join(save_folder_ls[1], "model_test.pkl"), "wb"),
    )

Preparing function arguments...
Calculating DWPCs...


100%|█████████████████████████████████████████| 251/251 [02:24<00:00,  1.74it/s]



Reshaping Result Matrices...


100%|█████████████████████████████████████████| 251/251 [00:13<00:00, 17.97it/s]


Stacking columns...
Shaping Matricies...
Getting Target Values


100%|████████████████████████████| 5728635/5728635 [00:05<00:00, 1088013.54it/s]


Training Model
[Pipeline] ....... (step 1 of 3) Processing transformer, total=   6.0s
[Pipeline] ...... (step 2 of 3) Processing maxabs_scale, total=   2.2s
[Pipeline] ............. (step 3 of 3) Processing e_net, total=  21.2s
Getting Model Coef


### pass in holdout set to the generated model

#### look at the holdout

In [168]:
edges_test.head()

,start_id,type,end_id
0,CHEBI:135735,indication_CiD,DOID:10763
1,CHEBI:135738,indication_CiD,DOID:10763
2,CHEBI:135876,indication_CiD,DOID:11054
3,CHEBI:135923,indication_CiD,DOID:14499
4,CHEBI:135925,indication_CiD,DOID:1094


In [169]:
edges_test.tail()

,start_id,type,end_id
532,CHEBI:9667,indication_CiG,NCBIGene:367
533,CHEBI:41423,indication_CiG,NCBIGene:367
534,CHEBI:9168,indication_CiG,NCBIGene:7490
535,CHEBI:41879,indication_CiG,NCBIGene:367
536,CHEBI:8378,indication_CiG,NCBIGene:367


#### filter holdout set first
* start must be a compound
* end must be a disease. Metatype must be of the same type

In [170]:
# check the relation types
edges_test["type"].value_counts()

type
indication_CiD    521
indication_CiG     11
indication_CiP      5
Name: count, dtype: int64

In [176]:
# keep only end_ids that are diseases
edges_test_filt = edges_test.query('type == "indication_CiD"')

### Make predictions

In [177]:
if os.path.exists(os.path.join(save_folder_ls[1], "results_test.csv")):
    next
else:
    (pairs, out_mp), dwpc = mg_ls[1].extract_dwpc(
        metapaths=mp_to_extract,
        start_nodes=edges_test_filt.start_id.to_numpy(),
        end_nodes=edges_test_filt.end_id.to_numpy(),
        # start_nodes=test_comps_ls[i],
        # end_nodes=test_dis_ls[i],
        return_sparse=True,
        sparse_df=False,
        verbose=True,
        n_jobs=30,
    )

    print("Getting Target Values")
    y = []
    for row in tqdm(pairs.itertuples(), total=len(pairs)):
        # pos_tups defined several cells above
        if set([(row.chemicalsubstance_id, row.disease_id)]) & pos_tups_ls[1]:
            y.append(1)
        else:
            y.append(0)

    # Fit the model and get results
    print("Getting Probabilities")
    y_proba = post_extraction_pipeline.predict_proba(dwpc)[:, 1]

    # Get metrics
    roc_auc = roc_auc_score(y, y_proba)
    avg_prec = average_precision_score(y, y_proba)
    print("AUROC: {:1.4f}".format(roc_auc))
    print("AUPR: {:1.4f}".format(avg_prec))

    pairs["status"] = y
    pairs["proba"] = y_proba

    # Save results
    pairs.to_csv(os.path.join(save_folder_ls[1], "results_test.csv"), index=False)

Preparing function arguments...
Calculating DWPCs...


100%|█████████████████████████████████████████| 251/251 [02:50<00:00,  1.47it/s]



Reshaping Result Matrices...


100%|████████████████████████████████████████| 251/251 [00:02<00:00, 108.31it/s]


Stacking columns...
Getting Target Values


100%|██████████████████████████████| 271441/271441 [00:00<00:00, 1139616.06it/s]


Getting Probabilities
AUROC: 0.8038
AUPR: 0.1054


### Output
* `status` represents whether the compound-disease pair is an indication (1) or not (0).
* `proba` is the score of the compound-disease pair.
* compound-disease pairs that have a lower `proba` are less likely to be true than those with a higher `proba`

In [182]:
# import the results back in
results = pd.read_csv(
    os.path.join(save_folder_ls[1], "results_test.csv")
).drop_duplicates()

In [183]:
# get a view of the dataframe
results.head()

,chemicalsubstance_id,disease_id,status,proba
0,CHEBI:135735,DOID:0050425,0,0.007140
4,CHEBI:135735,DOID:0050742,0,0.006813
5,CHEBI:135735,DOID:0050783,0,0.008267
6,CHEBI:135735,DOID:0060224,0,0.018347
10,CHEBI:135735,DOID:0080159,0,0.006699
